Logistic Regression

In [1]:
from logistic_regression import *

### Importing HuggingFace

In [2]:
train_dataset = load_dataset('imdb', split='train')
test_dataset = load_dataset('imdb', split='test')

Reusing dataset imdb (/home/surenis/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (/home/surenis/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


In [3]:
x_train, y_train = generate_x_and_y(train_dataset)
x_test, y_test = generate_x_and_y(test_dataset)

### Logistic Regression

Let's search for all the words finishing with !

As it would make no sense that the ! character appears before the end of the word

it appears no words ends with ! in the training set

Thus, there is no need to check for ! in the documents

In [4]:
X_train_features = LoRegression(x_train, y_train)

100%|██████████| 25000/25000 [15:08<00:00, 27.50it/s]


In [5]:
clf = LogisticRegression(random_state=0).fit(X_train_features, y_train)

In [6]:
#create X_test using loRegression to have a usable informations
X_test_features = LoRegression(x_test, y_test)

100%|██████████| 25000/25000 [15:08<00:00, 27.51it/s]


In [7]:
y_pred = clf.predict(X_test_features)
print(X_test_features.shape, y_pred.shape)

(25000, 8) (25000,)


In [8]:
clf.score(X_test_features, y_test)

0.71532

In [9]:
from sklearn.metrics import classification_report

target_names = ['negative review', 'positive review']
print(classification_report(y_test, y_pred, target_names=target_names))

                 precision    recall  f1-score   support

negative review       0.71      0.73      0.72     12500
positive review       0.72      0.70      0.71     12500

       accuracy                           0.72     25000
      macro avg       0.72      0.72      0.72     25000
   weighted avg       0.72      0.72      0.72     25000



In [16]:
acc = ""
for word in x_test[101]:
    acc = acc + word + " "
print(acc)

i must say I am an avid horror movie fan, and currently i cannot get enough of foreign horror. since us horror really depends too much on gore.this movie is fantastic. this movie reminded me a lot of m. night's sixth sense. the way the film was directed was great. the director took his time to set everything up. it took about a good 40 mins just to set the movie up into a horror movie. i thought the movie was just a drama. this movie just builds up and the pay off is not too much.I have never done this before, but this is the main reason i liked the movie.... i actually screamed out loud because one scene just startled the hell out of me... which is kind of pathetic.... but for a film to do that, it is great. 


In [17]:
print("Review was labelled as: ",y_pred[101],"instead of: ",y_test[101])

Review was labelled as:  0 instead of:  1


In [18]:
X_test_features[101]

array([ 0.        ,  0.        ,  6.        ,  4.95582706,  5.        ,
       -8.7       ,  4.        ,  7.3       ])